In [82]:
from typing import TypedDict, List, Dict, Any, Optional
from typing_extensions import NotRequired

In [83]:
class TrelloSlackState(TypedDict):
    """
    State passing between the nodes of the network.
    NotRequired means that these fields may not be present initially.
    """
    # User Input
    input: str
    # Input source (Channel ID)
    channel_id: str
    # Procesed input answer
    intent: NotRequired[str]
    details: NotRequired[Dict[str, any]]
    # Trello return
    trello_result: NotRequired[Dict[str, Any]]
    # Answer message
    response: NotRequired[str]
    # Flow control
    error: NotRequired[str]
    next: NotRequired[str]

In [84]:
from langchain.agents import AgentExecutor
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
import os

In [ ]:
# Loading enviroments constants
load_dotenv()
TRELLO_API_KEY = os.getenv("TRELLO_API_KEY")
TRELLO_TOKEN = os.getenv("TRELLO_TOKEN")
SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
SLACK_CHANNEL_ID = os.getenv("SLACK_CHANNEL_ID")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
APP_TOKEN = os.getenv("APP_TOKEN")

In [86]:
import requests
import json
import re

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

In [87]:
gemini = ChatGoogleGenerativeAI(
    model="models/gemini-2.0-flash",
    google_api_key=GEMINI_API_KEY,
    temperature=0.7,
    max_tokens = 8000
)

gemini.invoke('hola que tal')

AIMessage(content='¡Hola! ¿Qué tal tú? ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ea9adac6-52f0-4671-a13a-6495583d6316-0', usage_metadata={'input_tokens': 3, 'output_tokens': 16, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}})

In [88]:
def parse_user_input(user_message: str) -> tuple:
    """
    Analyzes the user message to extract intent and details.
    Returns: (intent, details)
    """
    prompt = f"""
    Analyze the user message and extract the intent and details:

    Message: {user_message}

    IMPORTANT: Your response MUST be valid JSON with no additional text.
    Answer EXACTLY in this format: {{"intent": "action_type", "details": {{"key": "value"}}}}
    For example: {{"intent": "move_card", "details": {{"card_name": "Bug fix", "source_list": "In Progress", "target_list": "Done"}}}}
    """
    
    # Create a human message for Langchain
    message = HumanMessage(content=prompt)
    
    # Get response from Gemini
    response = gemini.invoke([message])
    response_text = response.content
    
    # Debug lines - uncomment if needed
    # print(f'Model response: {response_text}')
    
    # Try to extract JSON from the response
    try:
        # Find the first { and the last } to extract only the JSON
        start = response_text.find('{')
        end = response_text.rfind('}') + 1
        
        if start >= 0 and end > start:
            json_str = response_text[start:end]
            response_dict = json.loads(json_str)
            intent = response_dict['intent']
            details = response_dict['details']
            return intent, details
        else:
            print("No JSON format found in the response")
            return None, None
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return None, None

In [89]:
from langgraph.graph import StateGraph

def parse_message_node(state: TrelloSlackState) -> TrelloSlackState:
    """
    Node that analyzes the message and extracts intent and details.
    Updates the state with the detected intent.
    """
    user_message = state["input"]
    
    try:
        # Call existing function
        intent, details = parse_user_input(user_message)
        
        if intent is None:
            return {"error": "Could not determine message intent", "next": "handle_error"}
        
        # Update state with extracted information
        return {
            "intent": intent,
            "details": details,
            "next": "trello_actions"  # Indicates that the next node should be get_trello_info
        }
    except Exception as e:
        return {"error": f"Error processing message: {str(e)}", "next": "handle_error"}

# Testing so far

In [90]:
def test_parse_message():
    """Function to test the message analysis node."""
    # Example messages to test
    test_messages = [
        "Move card 'Bug fix' from 'In Progress' to 'Done'",
        "Create a new card called 'Update documentation' in the 'Pending' list",
        "Show me all cards in the 'In Progress' list",
        "Generate a daily activity report"
    ]
    
    for message in test_messages:
        print(f"\nTesting message: '{message}'")
        
        # Initial state
        initial_state = {"input": message}
        
        # Execute the node
        updated_state = parse_message_node(initial_state)
        
        # Show results
        print(f"Detected intent: {updated_state.get('intent')}")
        print(f"Details: {updated_state.get('details')}")
        print(f"Next node: {updated_state.get('next')}")
        print("-" * 50)

# Run the test if this file is the main one
if __name__ == "__main__":
    test_parse_message()


Testing message: 'Move card 'Bug fix' from 'In Progress' to 'Done''
Detected intent: move_card
Details: {'card_name': 'Bug fix', 'source_list': 'In Progress', 'target_list': 'Done'}
Next node: trello_actions
--------------------------------------------------

Testing message: 'Create a new card called 'Update documentation' in the 'Pending' list'
Detected intent: create_card
Details: {'card_name': 'Update documentation', 'list_name': 'Pending'}
Next node: trello_actions
--------------------------------------------------

Testing message: 'Show me all cards in the 'In Progress' list'
Detected intent: show_cards
Details: {'list_name': 'In Progress'}
Next node: trello_actions
--------------------------------------------------

Testing message: 'Generate a daily activity report'
Detected intent: generate_report
Details: {'report_type': 'daily activity'}
Next node: trello_actions
--------------------------------------------------


# Codigo Trello

In [91]:
# Trello API functions
def get_trello_boards():
    """
    Gets all Trello boards accessible to the user.
    Returns a dictionary mapping board names to board IDs.
    """
    url = "https://api.trello.com/1/members/me/boards"
    
    headers = {
        "Accept": "application/json"
    }
    
    params = {
        "key": TRELLO_API_KEY, 
        "token": TRELLO_TOKEN
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        # Create a dictionary of board names to board IDs
        boards_dict = {board['name']: board['id'] for board in response.json()}
        return boards_dict
    else:
        print(f"Error: {response.status_code}")
        return None

def get_trello_lists(board_id):
    """
    Gets all lists on a specific Trello board.
    Returns a dictionary mapping list names to list IDs.
    """
    url = f"https://api.trello.com/1/boards/{board_id}/lists"
    
    params = {
        "key": TRELLO_API_KEY, 
        "token": TRELLO_TOKEN
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        lists_dict = {list_item['name']: list_item['id'] for list_item in response.json()}
        return lists_dict
    else:
        print(f"Error: {response.status_code}")
        return None

def get_cards_in_list(list_id):
    """
    Gets all cards in a specific Trello list.
    Returns a dictionary mapping card names to card IDs.
    """
    url = f"https://api.trello.com/1/lists/{list_id}/cards"
    
    headers = {
        "Accept": "application/json"
    }
    
    params = {
        "key": TRELLO_API_KEY, 
        "token": TRELLO_TOKEN
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        card_dict = {card['name']: card['id'] for card in response.json()}
        return card_dict
    else:
        print(f"Error: {response.status_code}")
        return None

def create_trello_card(list_id, name, desc=""):
    """
    Creates a new card in the specified list.
    """
    url = "https://api.trello.com/1/cards"
    
    headers = {
        "Accept": "application/json"
    }
    
    params = {
        "idList": list_id,
        "name": name,
        "desc": desc,
        "key": TRELLO_API_KEY,
        "token": TRELLO_TOKEN,
        "pos": "top"
    }
    
    response = requests.post(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

def update_trello_card(card_id, list_id=None, name=None, desc=None):
    """
    Updates a Trello card. Any parameter that is None will not be updated.
    """
    url = f"https://api.trello.com/1/cards/{card_id}"
    
    headers = {
        "Accept": "application/json"
    }
    
    params = {
        "key": TRELLO_API_KEY,
        "token": TRELLO_TOKEN
    }
    
    # Only add parameters that are not None
    if list_id:
        params["idList"] = list_id
    if name:
        params["name"] = name
    if desc:
        params["desc"] = desc
    
    response = requests.put(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

def get_trello_card(card_id):
    """
    Gets details of a specific Trello card.
    """
    url = f"https://api.trello.com/1/cards/{card_id}"
    
    headers = {
        "Accept": "application/json"
    }
    
    params = {
        "key": TRELLO_API_KEY,
        "token": TRELLO_TOKEN
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

In [92]:
from datetime import datetime

def generate_daily_summary(cards):
    """
    Generates a daily summary of activities based on Trello cards.
    
    Args:
        cards: List of card objects from Trello API
        
    Returns:
        A formatted string with the summary
    """
    # Get current date
    today = datetime.now().date()
    
    summary = "# Daily Stand-Up Summary\n\n"
    summary += f"Date: {today.strftime('%d/%m/%Y')}\n\n"
    
    # Filter cards updated today (optional)
    today_cards = []
    for card in cards:
        last_activity = datetime.fromisoformat(card['dateLastActivity'].replace('Z', '+00:00'))
        if last_activity.date() == today:
            today_cards.append(card)
    
    if not today_cards:
        summary += "No cards were updated today.\n"
        return summary
    
    summary += f"## Cards Updated Today ({len(today_cards)})\n\n"
    
    for card in today_cards:
        # Extract relevant information
        name = card['name']
        description = card['desc'] if card['desc'] else "No description"
        status = "Open" if not card['closed'] else "Closed"
        url = card['url']
        
        # Add to summary
        summary += f"### {name}\n"
        summary += f"- **Status:** {status}\n"
        summary += f"- **Description:** {description}\n"
        summary += f"- **Last Updated:** {card['dateLastActivity']}\n"
        summary += f"- **URL:** {url}\n\n"
    
    return summary

# Nodes

In [93]:
def trello_actions_node(state: TrelloSlackState) -> TrelloSlackState:
    """
    Node that performs Trello actions based on the detected intent.
    Updates the state with the result of the operation.
    """
    intent = state.get("intent")
    details = state.get("details", {})
    
    try:
        # Get lists from the specified board (or default board)
        board_id = details.get("board_id", BOARD_ID)
        lists = get_trello_lists(board_id)
        
        if not lists:
            return {
                "error": f"Could not retrieve lists from board {board_id}",
                "next": "handle_error"
            }
        
        # Handle different intents
        if intent == "show_cards":
            list_name = details.get("list_name")
            if not list_name:
                return {"error": "No list name specified", "next": "handle_error"}
            
            if list_name not in lists:
                return {"error": f"List '{list_name}' not found", "next": "handle_error"}
            
            cards = get_cards_in_list(lists[list_name])
            return {
                "trello_result": {
                    "type": "cards_list",
                    "list_name": list_name,
                    "cards": cards
                },
                "next": "format_response"
            }
            
        elif intent == "move_card":
            card_name = details.get("card_name")
            source_list = details.get("source_list")
            target_list = details.get("target_list")
            
            if not all([card_name, source_list, target_list]):
                return {"error": "Missing details for moving card", "next": "handle_error"}
            
            if source_list not in lists or target_list not in lists:
                return {"error": "Source or target list not found", "next": "handle_error"}
            
            # Get card ID from source list
            cards = get_cards_in_list(lists[source_list])
            if not cards or card_name not in cards:
                return {"error": f"Card '{card_name}' not found in '{source_list}'", "next": "handle_error"}
            
            # Move the card
            result = update_trello_card(card_id=cards[card_name], list_id=lists[target_list])
            return {
                "trello_result": {
                    "type": "card_moved",
                    "card_name": card_name,
                    "from_list": source_list,
                    "to_list": target_list,
                    "card_data": result
                },
                "next": "format_response"
            }
            
        elif intent == "create_card":
            card_name = details.get("card_name")
            list_name = details.get("list_name")
            description = details.get("description", "")
            
            if not all([card_name, list_name]):
                return {"error": "Missing details for creating card", "next": "handle_error"}
            
            if list_name not in lists:
                return {"error": f"List '{list_name}' not found", "next": "handle_error"}
            
            # Create the card
            result = create_trello_card(list_id=lists[list_name], name=card_name, desc=description)
            return {
                "trello_result": {
                    "type": "card_created",
                    "card_name": card_name,
                    "list_name": list_name,
                    "card_data": result
                },
                "next": "format_response"
            }
            
        elif intent == "list_boards":
            # Get all boards
            boards = get_trello_boards()
            return {
                "trello_result": {
                    "type": "boards_list",
                    "boards": boards
                },
                "next": "format_response"
            }
            
        elif intent == "generate_report":
            report_type = details.get("report_type")
            
            if report_type == "daily activity":
                # Get all lists in the board
                all_cards = []
                
                for list_name, list_id in lists.items():
                    # Get all cards in this list
                    cards_in_list = get_cards_in_list(list_id)
                    if cards_in_list:
                        # For each card, get its details
                        for card_name, card_id in cards_in_list.items():
                            card_details = get_trello_card(card_id)
                            if card_details:
                                all_cards.append(card_details)
                
                # Generate the summary
                summary = generate_daily_summary(all_cards)
                
                return {
                    "trello_result": {
                        "type": "report",
                        "report_type": report_type,
                        "message": summary
                    },
                    "next": "format_response"
                }
            else:
                return {"error": f"Unknown report type: {report_type}", "next": "handle_error"}
        
        else:
            return {"error": f"Unknown intent: {intent}", "next": "handle_error"}
    
    except Exception as e:
        return {"error": f"Error in Trello operations: {str(e)}", "next": "handle_error"}

In [100]:
def format_response_node(state: TrelloSlackState) -> TrelloSlackState:
    """
    Node that formats the Trello result into a readable message.
    Updates the state with the formatted response.
    """
    trello_result = state.get("trello_result", {})
    result_type = trello_result.get("type")
    
    try:
        if result_type == "cards_list":
            list_name = trello_result.get("list_name")
            cards = trello_result.get("cards", {})
            
            if not cards:
                response = f"No cards found in list '{list_name}'."
            else:
                card_items = "\n".join([f"• {card_name}" for card_name in cards.keys()])
                response = f"📋 **Cards in '{list_name}':**\n\n{card_items}"
        
        elif result_type == "card_moved":
            card_name = trello_result.get("card_name")
            from_list = trello_result.get("from_list")
            to_list = trello_result.get("to_list")
            
            response = f"✅ Card '{card_name}' moved from '{from_list}' to '{to_list}'."
        
        elif result_type == "card_created":
            card_name = trello_result.get("card_name")
            list_name = trello_result.get("list_name")
            
            response = f"✅ Created new card '{card_name}' in list '{list_name}'."
        
        elif result_type == "boards_list":
            boards = trello_result.get("boards", {})
            
            if not boards:
                response = "No Trello boards found."
            else:
                board_items = "\n".join([f"• {board_name}" for board_name in boards.keys()])
                response = f"📋 **Your Trello Boards:**\n\n{board_items}"
        
        elif result_type == "report":
            report_type = trello_result.get("report_type")
            message = trello_result.get("message", "")
            
            # For daily activity reports, the message is already formatted
            if report_type == "daily activity":
                response = message
            else:
                response = f"📊 **{report_type.title()} Report**\n\n{message}"
        
        else:
            response = f"Operation completed successfully: {result_type}"
        
        return {
            "response": response,
            "next": "send_to_slack"
        }
    
    except Exception as e:
        return {"error": f"Error formatting response: {str(e)}", "next": "handle_error"}

In [95]:
def handle_error_node(state: TrelloSlackState) -> TrelloSlackState:
    """
    Node that handles errors and generates appropriate error messages.
    """
    error_msg = state.get("error", "An unknown error occurred")
    
    response = f"❌ **Error:** {error_msg}"
    
    return {
        "response": response,
        "next": "send_to_slack"
    }

In [96]:
def send_to_slack_node(state: TrelloSlackState) -> TrelloSlackState:
    """
    Node that sends the response to Slack.
    """
    response = state.get("response", "")
    channel_id = state.get("channel_id", SLACK_CHANNEL_ID)
    
    try:
        # Send message to Slack
        url = "https://slack.com/api/chat.postMessage"
        headers = {"Authorization": f"Bearer {SLACK_BOT_TOKEN}"}
        payload = {"channel": channel_id, "text": response}
        
        slack_response = requests.post(url, headers=headers, json=payload)
        
        # Check Slack API response
        response_data = slack_response.json()
        if slack_response.status_code == 200 and response_data.get("ok"):
            print(f"Message sent successfully to Slack")
            return {
                "response": response,
                "slack_result": response_data
            }
        else:
            error_msg = f"Error sending message to Slack: {response_data}"
            print(error_msg)
            return {
                "error": error_msg,
                "response": response
            }
    
    except Exception as e:
        error_msg = f"Exception sending message to Slack: {str(e)}"
        print(error_msg)
        return {
            "error": error_msg,
            "response": response
        }

In [97]:
from langgraph.graph import StateGraph, END

def build_trello_slack_graph():
    """
    Builds the complete graph for the Trello-Slack agent.
    """
    # Initialize the graph
    graph = StateGraph(TrelloSlackState)
    
    # Add all nodes
    graph.add_node("parse_message", parse_message_node)
    graph.add_node("trello_actions", trello_actions_node)
    graph.add_node("format_response", format_response_node)
    graph.add_node("handle_error", handle_error_node)
    graph.add_node("send_to_slack", send_to_slack_node)
    
    # Set the entry point
    graph.set_entry_point("parse_message")
    
    # Add conditional edges based on the 'next' field
    # From parse_message node
    graph.add_conditional_edges(
        "parse_message",
        lambda state: state.get("next", "trello_actions"),
        {
            "trello_actions": "trello_actions",
            "handle_error": "handle_error"
        }
    )
    
    # From trello_actions node
    graph.add_conditional_edges(
        "trello_actions",
        lambda state: state.get("next", "format_response"),
        {
            "format_response": "format_response",
            "handle_error": "handle_error"
        }
    )
    
    # From format_response node
    graph.add_edge("format_response", "send_to_slack")
    
    # From handle_error node
    graph.add_edge("handle_error", "send_to_slack")
    
    # From send_to_slack node (end of the graph)
    graph.add_edge("send_to_slack", END)
    
    # Compile the graph
    return graph.compile()

In [98]:
def process_slack_message(message, channel_id=None):
    """
    Processes a Slack message through the LangGraph agent.
    """
    # Build the graph
    graph = build_trello_slack_graph()
    
    # Set the initial state
    initial_state = {
        "input": message,
        "channel_id": channel_id or SLACK_CHANNEL_ID
    }
    
    # Execute the graph
    try:
        result = graph.invoke(initial_state)
        return result
    except Exception as e:
        error_msg = f"Error executing graph: {str(e)}"
        print(error_msg)
        return {"error": error_msg}

In [99]:
def test_full_graph():
    """
    Tests the full LangGraph with different message examples.
    """
    # Example messages to test
    test_messages = [
        "Show me all cards in the 'In Progress' list",
        "Move card 'Bug fix' from 'In Progress' to 'Done'",
        "Create a new card called 'Update documentation' in the 'Pending' list",
        "Generate a daily activity report",
        "List all boards"
    ]
    
    for i, message in enumerate(test_messages):
        print(f"\n\n==== TEST {i+1}: '{message}' ====\n")
        
        # Process the message
        result = process_slack_message(message)
        
        # Print the result
        print("RESULT:")
        for key, value in result.items():
            # Skip printing large objects
            if isinstance(value, dict) and len(str(value)) > 100:
                print(f"{key}: [Large object with {len(value)} items]")
            else:
                print(f"{key}: {value}")

# Run the test if this file is the main one
if __name__ == "__main__":
    test_full_graph()



==== TEST 1: 'Show me all cards in the 'In Progress' list' ====

Message sent successfully to Slack
RESULT:
input: Show me all cards in the 'In Progress' list
channel_id: C08RL4K9YNN
intent: show_cards
details: {'list_name': 'In Progress'}
trello_result: {'type': 'cards_list', 'list_name': 'In Progress', 'cards': {}}
response: No cards found in list 'In Progress'.
next: send_to_slack


==== TEST 2: 'Move card 'Bug fix' from 'In Progress' to 'Done'' ====

Message sent successfully to Slack
RESULT:
input: Move card 'Bug fix' from 'In Progress' to 'Done'
channel_id: C08RL4K9YNN
intent: move_card
details: {'card_name': 'Bug fix', 'source_list': 'In Progress', 'target_list': 'Done'}
response: ❌ **Error:** Card 'Bug fix' not found in 'In Progress'
error: Card 'Bug fix' not found in 'In Progress'
next: send_to_slack


==== TEST 3: 'Create a new card called 'Update documentation' in the 'Pending' list' ====

Message sent successfully to Slack
RESULT:
input: Create a new card called 'Update d